In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import json
import requests
import os
import re

In [3]:
from gdc_query import gdc_data

In [4]:
test = gdc_data("LIHC")

In [5]:
test.read_csv()

In [6]:
test.data.head()

,004c60cf-c08e-49df-b4ce-baca41e11250.htseq.counts,0069f64b-8d8f-4426-968d-23483929ee58.htseq.counts,014b9b85-3128-416b-93d4-7ace3b676d4e.htseq.counts,03011a57-3e95-49d1-a927-cff4111d2d5b.htseq.counts,0415a9b4-a58d-4641-ab1a-927ed7a04824.htseq.counts,047bd029-d63b-4f25-8a73-b95ad72d434f.htseq.counts,04dc4da1-1d1a-46da-a9d8-da9964591aec.htseq.counts,04e7f1a4-3173-4a6f-af60-04e1f2e29868.htseq.counts,05ac7b05-e459-4833-97fc-530185a7a55f.htseq.counts,05efced9-d60f-43ea-9b61-c6efaf902e7d.htseq.counts,...,fb94b719-4a43-4090-9a61-9f62a30d545d.htseq.counts,fbbb6d26-8bd4-40da-870b-d8db0f653cfa.htseq.counts,fca37687-75b4-4ca6-9963-36b468ca01a7.htseq.counts,fdb62f73-33a7-44c3-950c-739383b9dd30.htseq.counts,fe506b98-0733-43c9-943a-be9b12f1c2fb.htseq.counts,fe625352-dd2e-478d-8d21-06659f854945.htseq.counts,fe76a5ca-f70a-4ab7-b080-5a19ae36dc2b.htseq.counts,feae9113-b2f3-4dd4-9faf-6076eb32c925.htseq.counts,ff8776f1-5499-459c-989f-0bc5268e6631.htseq.counts,ffeed225-c2a3-4b4c-954c-4816903782a9.htseq.counts
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.13,2155,3871,5085,6070,3231,6762,2285,3395,6637,3560,...,33015,2853,9818,9805,2644,2874,2136,5630,2364,4941
ENSG00000000005.5,1,4,16,1,0,0,1,1,1,2,...,13,2,0,0,0,0,3,1,0,2
ENSG00000000419.11,405,1133,1326,1047,1130,1079,1105,1302,476,1174,...,1250,537,1390,1598,817,1026,569,1317,1087,1717
ENSG00000000457.12,170,727,587,327,638,834,585,853,184,461,...,574,58,535,910,465,814,304,1129,1428,1287
ENSG00000000460.15,49,167,1528,235,340,204,119,451,91,127,...,146,14,226,361,176,183,95,244,346,450


. Investigate methylation data

In [3]:
norm = "jhu-usc.edu_LIHC.HumanMethylation450.1.lvl-3.TCGA-BC-A10Q-11A-11D-A132-05.gdc_hg38.txt"
can = "jhu-usc.edu_LIHC.HumanMethylation450.1.lvl-3.TCGA-BC-A10Q-01A-11D-A132-05.gdc_hg38.txt"

meth_norm = pd.read_table(norm,sep="\t").fillna(0)
meth_can = pd.read_table(can,sep="\t").fillna(0)

In [4]:
meth = pd.concat([meth_norm.iloc[:,1],meth_can.iloc[:,1]],axis=1)
meth.shape

(485577, 2)

In [7]:
#Import packages for principle component analysis 
from sklearn.decomposition import PCA

In [8]:
pca = PCA(n_components=2)
#Everything except for the last 5 rows
pca.fit(meth.T)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [12]:
pc = pca.transform(meth.T) 

In [13]:
pc.shape

(2, 2)

In [7]:
from sklearn.decomposition import PCA

In [8]:
pca = PCA(n_components=3)
#Everything except for the last 5 rows
pca.fit(test.data.iloc[:-5,].T)

PCA(copy=True, n_components=3, whiten=False)

In [9]:
sum(pca.explained_variance_ratio_)

0.78030891808336023

In [10]:
pc = pca.transform(test.data.iloc[:-5,].T) 

In [11]:
pc = pd.DataFrame(pc)

In [12]:
pc.shape

(424, 3)

In [13]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [18]:
plt.plot(pc.iloc[:,0],pc.iloc[:,1],'x')
plt.show()

In [30]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pc.iloc[:,0],pc.iloc[:,1],pc.iloc[:,2],c=pc['color'])
plt.show()

In [5]:
cases.project.project_id in ["TCGA-LIHC"] and cases.samples.sample_type_id in ["11"] and 
files.analysis.workflow_type in ["HTSeq - Counts"] and files.experimental_strategy in ["RNA-Seq"]

In [9]:
files_endpt = "https://api.gdc.cancer.gov/files"
# Filters for identifying the file names for Solid Tissue Normal (STN)
# This set of filters is nested under an 'and' operator.
filt = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ["TCGA-LIHC"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.samples.sample_type_id",
            "value": ["11"]
            }
        },
        {
        "op": "in",
        "content":{
           "field": "files.analysis.workflow_type",
            "value": ["HTSeq - Counts"]
            }
        },
        {"op":"in",
        "content":{
            "field": "files.experimental_strategy",
            "value": ["RNA-Seq"]
            }
        }
    ]
}

params = {'filters':json.dumps(filt),
         'size': 1000,
         'fields': 'file_name' #Set to only return the file name 
         }
# requests URL-encodes automatically
response = requests.get(files_endpt, params = params)
#Solid Tissue Normal (stn), file names
stnJson = json.dumps(response.json(), indent=2)

In [13]:
json.loads(response.content.decode("utf-8"))

{'data': {'hits': [{'file_name': '2b8bf629-3c22-4dcb-a9a5-ec01c5099167.htseq.counts.gz',
    'id': 'c64bb408-1b1e-4a02-b796-c40654e4145c'},
   {'file_name': '4d3842d1-c643-4659-98e9-8d613144fa91.htseq.counts.gz',
    'id': '10de4282-01e0-4990-9f89-9cf643a7c9d1'},
   {'file_name': 'f43699d8-3acc-445d-8ab2-9475194ebbc9.htseq.counts.gz',
    'id': '8d8c295a-9a07-414e-8f4c-3a0f6575e2fe'},
   {'file_name': '9abf327b-d44f-435d-986d-09107bcd2643.htseq.counts.gz',
    'id': 'bead64ab-3233-49df-a254-660ebc4e8928'},
   {'file_name': 'bc4f3a78-39a4-4a92-ad83-df251c5bf994.htseq.counts.gz',
    'id': '2c229ae0-c08e-43fa-8661-4229157ee6b2'},
   {'file_name': 'fa7877bb-9e46-4ecb-8c47-501893ce2410.htseq.counts.gz',
    'id': 'c6fbc477-2757-4b55-86b6-49a7c651e97c'},
   {'file_name': '6892a129-ae07-49df-a018-30c63c4d3bd7.htseq.counts.gz',
    'id': '405b2663-9223-46c8-bcaf-41f0d5b13b2c'},
   {'file_name': '5f94c33f-588b-4b6a-9c13-4505b0f94403.htseq.counts.gz',
    'id': 'fd27d631-a58f-45a5-af6f-36753d84

In [23]:
#Parse the json file so it can be used as a dictionary
parsed_json = json.loads(stnJson)

In [24]:
num = parsed_json['data']['pagination']['total'] #Acquire the number of hits
seq = [] #Initialize empty list for file names
#Iterate through the list of dictionarys and append the file name to the list
for dic in parsed_json['data']['hits']:
    seq.append(dic['file_name'])

In [25]:
seq
seq_cut = seq
for i in range(50):
    seq_cut[i] = seq_cut[i][:-3]
seq_cut

['2b8bf629-3c22-4dcb-a9a5-ec01c5099167.htseq.counts',
 '4d3842d1-c643-4659-98e9-8d613144fa91.htseq.counts',
 'f43699d8-3acc-445d-8ab2-9475194ebbc9.htseq.counts',
 '9abf327b-d44f-435d-986d-09107bcd2643.htseq.counts',
 'bc4f3a78-39a4-4a92-ad83-df251c5bf994.htseq.counts',
 'fa7877bb-9e46-4ecb-8c47-501893ce2410.htseq.counts',
 '6892a129-ae07-49df-a018-30c63c4d3bd7.htseq.counts',
 '5f94c33f-588b-4b6a-9c13-4505b0f94403.htseq.counts',
 'ec8fb205-0d42-4a66-b8ec-5bdd5da8a23b.htseq.counts',
 'b275a5d0-b9fe-4838-adb0-34db16a246af.htseq.counts',
 '8bf84ab0-7a48-4d98-bfc0-88537003ecd0.htseq.counts',
 '2d40e20c-bee4-48f2-a8b0-cdf8dff75dc3.htseq.counts',
 'a9123cf0-c6c0-42e7-b546-f11dfac355ce.htseq.counts',
 'd2141823-4f44-42d5-ad67-08a28d31eb63.htseq.counts',
 '2cc2e3ce-68cd-4690-9fff-5ecf86c2f57a.htseq.counts',
 '576fda32-4e0c-414d-b54c-259f2503fc11.htseq.counts',
 '6e146a57-21b2-4587-b888-01ed03ea9498.htseq.counts',
 '6cd9b90b-0c6c-44bd-8536-5a93553a7730.htseq.counts',
 '5745ce68-bb15-4d52-9132-4b

In [36]:
test.data.head()

,004c60cf-c08e-49df-b4ce-baca41e11250.htseq.counts,0069f64b-8d8f-4426-968d-23483929ee58.htseq.counts,014b9b85-3128-416b-93d4-7ace3b676d4e.htseq.counts,03011a57-3e95-49d1-a927-cff4111d2d5b.htseq.counts,0415a9b4-a58d-4641-ab1a-927ed7a04824.htseq.counts,047bd029-d63b-4f25-8a73-b95ad72d434f.htseq.counts,04dc4da1-1d1a-46da-a9d8-da9964591aec.htseq.counts,04e7f1a4-3173-4a6f-af60-04e1f2e29868.htseq.counts,05ac7b05-e459-4833-97fc-530185a7a55f.htseq.counts,05efced9-d60f-43ea-9b61-c6efaf902e7d.htseq.counts,...,fb94b719-4a43-4090-9a61-9f62a30d545d.htseq.counts,fbbb6d26-8bd4-40da-870b-d8db0f653cfa.htseq.counts,fca37687-75b4-4ca6-9963-36b468ca01a7.htseq.counts,fdb62f73-33a7-44c3-950c-739383b9dd30.htseq.counts,fe506b98-0733-43c9-943a-be9b12f1c2fb.htseq.counts,fe625352-dd2e-478d-8d21-06659f854945.htseq.counts,fe76a5ca-f70a-4ab7-b080-5a19ae36dc2b.htseq.counts,feae9113-b2f3-4dd4-9faf-6076eb32c925.htseq.counts,ff8776f1-5499-459c-989f-0bc5268e6631.htseq.counts,ffeed225-c2a3-4b4c-954c-4816903782a9.htseq.counts
0,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.13,2155,3871,5085,6070,3231,6762,2285,3395,6637,3560,...,33015,2853,9818,9805,2644,2874,2136,5630,2364,4941
ENSG00000000005.5,1,4,16,1,0,0,1,1,1,2,...,13,2,0,0,0,0,3,1,0,2
ENSG00000000419.11,405,1133,1326,1047,1130,1079,1105,1302,476,1174,...,1250,537,1390,1598,817,1026,569,1317,1087,1717
ENSG00000000457.12,170,727,587,327,638,834,585,853,184,461,...,574,58,535,910,465,814,304,1129,1428,1287
ENSG00000000460.15,49,167,1528,235,340,204,119,451,91,127,...,146,14,226,361,176,183,95,244,346,450


In [42]:
apple = "String.counts.gz"

In [43]:
apple[:-3]

'String.counts'

In [57]:
test.data.columns

Index(['004c60cf-c08e-49df-b4ce-baca41e11250.htseq.counts',
       '0069f64b-8d8f-4426-968d-23483929ee58.htseq.counts',
       '014b9b85-3128-416b-93d4-7ace3b676d4e.htseq.counts',
       '03011a57-3e95-49d1-a927-cff4111d2d5b.htseq.counts',
       '0415a9b4-a58d-4641-ab1a-927ed7a04824.htseq.counts',
       '047bd029-d63b-4f25-8a73-b95ad72d434f.htseq.counts',
       '04dc4da1-1d1a-46da-a9d8-da9964591aec.htseq.counts',
       '04e7f1a4-3173-4a6f-af60-04e1f2e29868.htseq.counts',
       '05ac7b05-e459-4833-97fc-530185a7a55f.htseq.counts',
       '05efced9-d60f-43ea-9b61-c6efaf902e7d.htseq.counts',
       ...
       'fb94b719-4a43-4090-9a61-9f62a30d545d.htseq.counts',
       'fbbb6d26-8bd4-40da-870b-d8db0f653cfa.htseq.counts',
       'fca37687-75b4-4ca6-9963-36b468ca01a7.htseq.counts',
       'fdb62f73-33a7-44c3-950c-739383b9dd30.htseq.counts',
       'fe506b98-0733-43c9-943a-be9b12f1c2fb.htseq.counts',
       'fe625352-dd2e-478d-8d21-06659f854945.htseq.counts',
       'fe76a5ca-f70a-4ab7-b0

In [79]:
pc['label']

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
394    0
395    0
396    0
397    0
398    0
399    0
400    0
401    0
402    0
403    0
404    0
405    0
406    0
407    0
408    0
409    0
410    0
411    0
412    0
413    0
414    0
415    0
416    0
417    0
418    0
419    0
420    0
421    0
422    0
423    0
Name: label, dtype: int64

In [26]:
lap = np.in1d(test.data.columns,seq_cut)

In [27]:
lap

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True, False, False, False, False, False, False,  True,  True,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True,

In [28]:
pc['color'] = ["Blue" if x else "Red" for x in lap]

In [29]:
pc

,0,1,2,color
0,-2.296973e+06,-3.414047e+06,5.700028e+05,Red
1,-7.759751e+05,2.038060e+05,-9.005815e+05,Red
2,-2.441984e+05,-2.470902e+06,-2.718885e+06,Red
3,2.044551e+06,-3.217580e+05,-9.252919e+04,Red
4,3.009963e+06,1.369600e+06,3.763464e+04,Red
5,3.839479e+05,7.718369e+05,4.436665e+05,Red
6,-9.125394e+05,3.899077e+05,9.006378e+04,Red
7,1.291448e+06,2.947950e+05,2.888563e+05,Red
8,-2.114823e+06,-8.774298e+05,-1.189472e+06,Red
9,-2.162015e+06,-1.603421e+06,4.405083e+05,Red


In [141]:
plt.scatter(pc.iloc[:,0],pc.iloc[:,1],c=pc['color'])
plt.show()

KeyError: 'color'

## Simple Nucleotide Variation

In [79]:
muse = pd.read_table('./data/SNV/TCGA.KIRP.muse.somatic.maf',sep='\t')
mutect = pd.read_table('./data/SNV/TCGA.KIRP.mutect.somatic.maf',sep='\t')
sniper = pd.read_table('./data/SNV/TCGA.KIRP.somaticsniper.somatic.maf',sep='\t')
varscan = pd.read_table('./data/SNV/TCGA.KIRP.varscan.somatic.maf',sep='\t')

C:\Users\oaoni\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (88,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\oaoni\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
muse.columns.values

array(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build',
       'Chromosome', 'Start_Position', 'End_Position', 'Strand',
       'Variant_Classification', 'Variant_Type', 'Reference_Allele',
       'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Tumor_Sample_Barcode',
       'Matched_Norm_Sample_Barcode', 'Match_Norm_Seq_Allele1',
       'Match_Norm_Seq_Allele2', 'Tumor_Validation_Allele1',
       'Tumor_Validation_Allele2', 'Match_Norm_Validation_Allele1',
       'Match_Norm_Validation_Allele2', 'Verification_Status',
       'Validation_Status', 'Mutation_Status', 'Sequencing_Phase',
       'Sequence_Source', 'Validation_Method', 'Score', 'BAM_File',
       'Sequencer', 'Tumor_Sample_UUID', 'Matched_Norm_Sample_UUID',
       'HGVSc', 'HGVSp', 'HGVSp_Short', 'Transcript_ID', 'Exon_Number',
       't_depth', 't_ref_count', 't_alt_count', 'n_depth', 'n_ref_count',
       'n_alt_count', 'all_effects', 'Allele', 'Gene', 'Feature',
       'Feature_ty

In [69]:
test = muse[['One_Consequence','SYMBOL']]

In [82]:
mutect[mutect.Hugo_Symbol == 'MUC4'].Hugo_Symbol

1466     MUC4
3207     MUC4
4722     MUC4
6041     MUC4
7690     MUC4
7691     MUC4
8087     MUC4
11018    MUC4
13113    MUC4
13911    MUC4
14140    MUC4
15826    MUC4
17295    MUC4
18740    MUC4
19161    MUC4
19351    MUC4
20533    MUC4
Name: Hugo_Symbol, dtype: object

In [87]:
mutect[(mutect.Consequence == 'inframe_insertion') & (mutect.SYMBOL == 'MUC4')].Chromosome

1466     chr3
7690     chr3
8087     chr3
13113    chr3
14140    chr3
17295    chr3
18740    chr3
19161    chr3
20533    chr3
Name: Chromosome, dtype: object

In [94]:
mutect.SYMBOL.unique().shape

(11610,)

## Query miRNA from GDC

In [37]:
import numpy as np
import pandas as pd
import json
import requests
import os
import re
from io import StringIO
from io import BytesIO
import gzip
import tarfile

In [273]:
files_endpt = "https://api.gdc.cancer.gov/files"
cancer = "KIRP"
size = "2000"
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content":
                {
                    "field": "cases.project.project_id",
                    "value": ["TCGA-"+cancer]
                }
        },
        {
            "op": "in",
            "content":
                {
                    "field": "files.experimental_strategy",
                    "value": ["miRNA-Seq"]     
                }
        },
        {
            "op": "in",
            "content":
                {
                    "field": "files.data_type",
                    "value": ["miRNA Expression Quantification"]     
                }     
        }
    ]
}

# Here a GET is used, so the filter parameters should be passed as a JSON string.
params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": size  #Set to the first 10 files for development
}

response = requests.get(files_endpt, params = params)
file_uuid_list = []

# This step populates the download list with the file_ids from the previous query
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])
    
data_endpt = "https://api.gdc.cancer.gov/data"

params = {"ids": file_uuid_list}
#Acquire memory location of compressed data from the data portal
response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})

response_head_cd = response.headers["Content-Disposition"]
#Acquire the name of the file
file_name = re.findall("filename=(.+)", response_head_cd)[0]

In [287]:
#Initialize dataframe
df = pd.DataFrame()
#Uncompression of the data response
with BytesIO(response.content) as targz:
    #Open a tarfile with response content
    with tarfile.open(fileobj=targz) as tar:
        for member in tar.getmembers()[1:]: #Iterate through the members of the tarfile
            #extract,decode and concatenate member to the dataframe
            with StringIO(tar.extractfile(member).read().decode('utf-8')) as data:
                df = pd.concat([df,pd.read_table(data,sep="\t",usecols=['read_count']).
                                rename(columns={'read_count':member.name.split('/')[0]})],axis=1)

In [288]:
df.head()

,00a3ea3f-8582-434b-ab40-251c23f4390f,0201e7b9-1b2f-403a-a61a-9327ba804a3b,026bba55-a20c-4e59-8dd0-3399735a66fc,041b359e-f6a5-430a-af1c-3bf4a81a96b5,04940285-f506-4c42-bac6-5ec15df76b1b,04cd260d-9323-4fdc-a533-a5f339d4ab9b,0515f874-b9eb-4946-a13f-63dff0d00829,068a7812-9e44-462b-b82c-c90221edf039,06de7e37-fc4a-4c0f-87ae-4738bb2c52f5,07ee3740-a0e5-4383-bbfc-327c4270e4c2,...,f7441d0e-6a29-4e83-8f41-33b14c7d3187,fac035a4-9d12-4f00-88ba-57e90e087413,fb4550a2-1ea5-49f3-a638-810d883a1ba0,fc6bb6ff-bf59-400f-9848-68f88c3eb918,fd66b842-a990-45fb-b550-eeff6443ef95,fe6f599e-0ee3-4682-9fef-e4f130d26d13,fefa6e9c-d886-47bf-897d-6761644311a6,ff0ffaa6-443f-4841-bcdc-5ced514f6c81,ff3cd143-45d3-4e67-bc8b-244134a1ba0d,ff4bb18c-1c96-44c5-ae37-cceb310994ba
0,52120,21789,13559,15036,30339,29321,19767,56505,23882,11959,...,62881,18781,13421,41016,14019,90723,38157,30018,50828,63586
1,51855,21671,13543,15003,30077,29174,19489,56792,23879,11945,...,62647,18962,13513,41256,14063,90595,38473,29431,50905,64178
2,52029,21882,13943,15334,30246,29530,19511,56663,23964,11829,...,62826,19282,13524,41389,14025,91651,38161,30099,50819,64799
3,87294,23707,4485,9141,18354,16214,9024,70584,14215,9014,...,32834,7587,13229,17176,8739,68139,44349,39943,39830,19653
4,21756,4311,1883,3663,1183,5748,2733,15388,7465,2638,...,4734,943,11419,2582,5309,21060,15737,23718,10779,17012
